In [95]:
import torch
import torch.nn as nn

In [96]:
torch.cuda.is_available()

True

In [97]:
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)

## Perceptron

`퍼셉트론(Perceptron)`은 프랑크 로젠블라트(Frank Rosenblatt)가 1957년에 제안한 `초기 형태의 인공 신경망`

다수의 입력으로부터 하나의 결과를 내보내는 알고리즘

각 입력값과 그에 해당되는 가중치의 곱의 전체 합이 `임계치(threshold)`를 넘으면 `활성화 함수(Activation Function)`(여기에선 `계단 함수, Step function`)에 의해 특정 값을 출력함

$ if \displaystyle\sum_{i}^{n} W_ix_i + b \geq \theta \rightarrow y = 1 $

$ if \displaystyle\sum_{i}^{n} W_ix_i + b < \theta \rightarrow y = 0 $

`활성화 함수(Activation Function)`는 출력을 변형시키는 함수

계단 함수를 시그모이드 함수로 변경시 구조적으로 로지스틱 회귀와 같음

## 단층 퍼셉트론(Single-Layer Perceptron)

퍼셉트론은 구조에 따라 `단층 퍼셉트론(Single-Layer Perceptron)`, `다층 퍼셉트론(Multi Layer Perceptron)` 으로 구분된다

단층 퍼셉트론은 값을 보내는 단계과 값을 받아서 출력하는 두 단계로 이루어 지며, 각 단계를 `층(layer)`이라 한다.

따라서 각 층을 `입력층(input layer)`, `출력층(output layer)` 이라 한다.

단층 퍼셉트론으로 구현 가능한 예시로 AND, NAND, OR 게이트가 있다.

이 경우 입력은 2개 출력은 1개 이며, 따라서 가중치도 2개이다.

가중치 편향 조합을 계산해서 넣으면 작동함을 볼 수 있다.

In [98]:
class gate:
    def __init__(self, w1, w2, b):
        self.w1 = w1
        self.w2 = w2
        self.b = b

    def step(self, input_value):
        return int(input_value >= 0)

    def gate(self, x1, x2):
        return self.step(x1*self.w1 + x2*self.w2 + self.b)

In [99]:
# AND
AND_gate = gate(0.5, 0.5, -0.7)
print(AND_gate.gate(0,0), AND_gate.gate(0,1), AND_gate.gate(1,0), AND_gate.gate(1,1))

0 0 0 1


In [100]:
# NAND
NAND_gate = gate(-0.5, -0.5, 0.7)
print(NAND_gate.gate(0,0), NAND_gate.gate(0,1), NAND_gate.gate(1,0), NAND_gate.gate(1,1))

1 1 1 0


In [101]:
# OR
OR_gate = gate(0.6, 0.6, -0.5)
print(OR_gate.gate(0,0), OR_gate.gate(0,1), OR_gate.gate(1,0), OR_gate.gate(1,1))

0 1 1 1


XOR 게이트는 단층 퍼셉트론으로 구현이 불가능하다.

수학적 구조상 단층 퍼셉트론은 두 영역을 구분하는 직선을 찾는것인데, XOR 게이트의 값은 하나의 직선으로 두 영역을 분리 할 수 없기 때문이다.

이는 선형 영역으로 분리해서 발생하는 문제로 비선형 영역으로 분리하면 구현 가능하다

In [102]:
# XOR 입출력
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to('cuda')
Y = torch.FloatTensor([[0], [1], [1], [0]]).to('cuda')

In [103]:
# 활성화 함수로 시그모이드 사용
linear = nn.Linear(2, 1, bias=True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid).to('cuda')

In [104]:
# 0, 1 을 분류하는 이진 분류 문제이므로 이진크로스엔트로피 함수 사용
# 비용 함수와 옵티마이저 정의
criterion = torch.nn.BCELoss().to('cuda')
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [105]:
# 비용이 줄어들지 않음을 볼 수 있음
for step in range(10001): 
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0: # 100번째 에포크마다 비용 출력
        print(step, cost.item())

0 0.7018222212791443
100 0.6931473016738892
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996
1000 0.6931471824645996
1100 0.6931471824645996
1200 0.6931471824645996
1300 0.6931471824645996
1400 0.6931471824645996
1500 0.6931471824645996
1600 0.6931471824645996
1700 0.6931471824645996
1800 0.6931471824645996
1900 0.6931471824645996
2000 0.6931471824645996
2100 0.6931471824645996
2200 0.6931471824645996
2300 0.6931471824645996
2400 0.6931471824645996
2500 0.6931471824645996
2600 0.6931471824645996
2700 0.6931471824645996
2800 0.6931471824645996
2900 0.6931471824645996
3000 0.6931471824645996
3100 0.6931471824645996
3200 0.6931471824645996
3300 0.6931471824645996
3400 0.6931471824645996
3500 0.6931471824645996
3600 0.6931471824645996
3700 0.6931471824645996
3800 0.6931471824645996
3900 0.6931471824645996
4000 0.6931471824645996
4100 0.6931471824645996
4200

In [106]:
# 단층 퍼셉트론으론 XOR 문제를 풀 수 없음을 볼 수 있음
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): \n', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): \n', predicted.detach().cpu().numpy())
    print('실제값(Y): \n', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis): 
 [[0.5]
 [0.5]
 [0.5]
 [0.5]]
모델의 예측값(Predicted): 
 [[0.]
 [0.]
 [0.]
 [0.]]
실제값(Y): 
 [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  0.5


## 다층 퍼셉트론(MultiLayer Perceptron, MLP)

XOR 게이트를 기존의 AND, NAND, OR 게이트를 조합하면 만들 수 있듯이, 퍼셉트론도 여러 층을 쌓으면 단층의 문제를 해결 가능하다

단층 퍼셉트론은 입력층과 출력층만 존재하지만, 다층 퍼셉트론은 입력층과 출력층 사이에 존재하는 `은닉층(hidden layer)`이 추가된다.

다층 퍼셉트론은 본래 은닉층이 1개 이상인 퍼셉트론을 뜻하지만 은닉층의 개수는 2개일 수도 있고, 수십 개일수도 있고 사용자가 설정하기 나름

은닉층이 2개 이상인 신경망을 `심층 신경망(Deep Neural Network, DNN)` 

심층 신경망은 다층 퍼셉트론만 이야기 하는 것이 아니라, 여러 변형된 다양한 신경망들도 은닉층이 2개 이상이 되면 심층 신경망이라 함

가중치를 스스로 찾아내도록 자동화시켜야하는데, 이것이 머신 러닝에서 말하는 `학습(training)` 단계에 해당

손실 함수(Loss function)와 옵티마이저(Optimizer)를 사용

심층 신경망을 학습시키는 경우, `딥 러닝(Deep Learning)` 이라 함

In [107]:
# XOR 입출력
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to('cuda')
Y = torch.FloatTensor([[0], [1], [1], [0]]).to('cuda')

In [108]:
# 은닉층이 3개인 인공신경망
model = nn.Sequential(
        nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
        nn.Sigmoid(),
        nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
        nn.Sigmoid(),
        nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
        nn.Sigmoid(),
        nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
        nn.Sigmoid()
        ).to('cuda')

In [109]:
# 이진 분류 이므로 이진크로스엔트로피 함수를 비용함수로 사용
criterion = torch.nn.BCELoss().to('cuda')
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1

In [110]:
for epoch in range(10001):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())

0 0.7555788159370422
100 0.6931402683258057
200 0.6931398510932922
300 0.6931394934654236
400 0.6931390762329102
500 0.693138599395752
600 0.6931382417678833
700 0.6931377649307251
800 0.6931372880935669
900 0.6931368112564087
1000 0.6931363344192505
1100 0.6931357979774475
1200 0.6931352615356445
1300 0.6931347250938416
1400 0.693134069442749
1500 0.6931334733963013
1600 0.693132758140564
1700 0.6931321620941162
1800 0.6931313276290894
1900 0.693130612373352
2000 0.6931297779083252
2100 0.6931288242340088
2200 0.6931278705596924
2300 0.693126916885376
2400 0.69312584400177
2500 0.6931246519088745
2600 0.693123459815979
2700 0.693122148513794
2800 0.6931207180023193
2900 0.6931191682815552
3000 0.6931175589561462
3100 0.693115770816803
3200 0.6931138038635254
3300 0.6931116580963135
3400 0.693109393119812
3500 0.6931068301200867
3600 0.6931039690971375
3700 0.6931008100509644
3800 0.6930972933769226
3900 0.6930934190750122
4000 0.6930890083312988
4100 0.6930840611457825
4200 0.69307839

In [111]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): \n', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): \n', predicted.detach().cpu().numpy())
    print('실제값(Y): \n', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis): 
 [[1.9882395e-04]
 [9.9977750e-01]
 [9.9982017e-01]
 [2.1199770e-04]]
모델의 예측값(Predicted): 
 [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y): 
 [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0
